In [1]:
%pip install torch_geometric gensim pypot annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [1]:
import os
import re
import time

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch import nn

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import gensim.downloader as api
from gensim.models import KeyedVectors

import networkx as nx
from torch_geometric.utils import to_networkx, coalesce, degree, to_undirected, dropout_adj

import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
drivestorage = "/content/drive/MyDrive/Colab Notebooks/data/"
def getDriveStorage(dir):
  return drivestorage + dir

In [ ]:
# Carrega o arquivo CSV em um DataFrame
#A proporção é 80% treino 20 teste.
df_test = pd.read_csv("data/raw/fnd/test.csv",sep=";")
df_train = pd.read_csv("data/raw/fnd/train.csv",sep=";")

In [ ]:
# Exibe as primeiras linhas do DataFrame
print(df_test.head())
print(df_train.head())

                                                text  label
0  trspnc park bucs win lose four playoff dang re...      0
1  whatever happen type meanwhile m rubin gestate...      1
2  review departure watch flirt death photo sad t...      1
3  ellen degeneres say never host president trump...      0
4  yemen war turn child young bride soldier accor...      1
                                                text  label
0  trump tell million american hurricane victim r...      0
1  new york cable provider altice acc network new...      0
2  japan dentsu get small fine overtime breach de...      1
3  little kid toxic stress solve priscilla chan c...      1
4  fantasy football kicker ranking week warn some...      0


# Pre-Processamento

In [ ]:
print(df_train.isnull().sum(),
df_train['label'].value_counts(normalize=True),
df_train['text'].str.split().apply(len).mean()
     )

text     0
label    0
dtype: int64 label
0    0.5
1    0.5
Name: proportion, dtype: float64 287.7889221556886


In [ ]:
df_train['text'].str.lower()
df_test['text'].str.lower()
#strip accents
df_train['text'] = df_train['text'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_test['text'] = df_test['text'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
df_train.to_parquet(getDriveStorage("interim/train_fnd_clean.parquet"))
df_test.to_parquet(getDriveStorage("interim/test_fnd_clean.parquet"))

In [4]:
df_train = pd.read_parquet(getDriveStorage("interim/train_fnd_clean.parquet"))
df_test = pd.read_parquet(getDriveStorage("interim/test_fnd_clean.parquet"))

# Exibe as primeiras linhas do DataFrame
print(df_test.head())
print(df_train.head())

                                                text  label
0  trspnc park bucs win lose four playoff dang re...      0
1  whatever happen type meanwhile m rubin gestate...      1
2  review departure watch flirt death photo sad t...      1
3  ellen degeneres say never host president trump...      0
4  yemen war turn child young bride soldier accor...      1
                                                text  label
0  trump tell million american hurricane victim r...      0
1  new york cable provider altice acc network new...      0
2  japan dentsu get small fine overtime breach de...      1
3  little kid toxic stress solve priscilla chan c...      1
4  fantasy football kicker ranking week warn some...      0


In [5]:
def preprocess(text):
    # tudo minúsculo, só letras/números
    text = text.lower()
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    text = re.sub(r"\s+", " ", text, flags=re.UNICODE).strip()
    tokens = text.split()
    tokens = [t for t in tokens if t not in ENGLISH_STOP_WORDS and len(t) > 2]
    return tokens

def document_vector(text, embeddings_index, dim=100):
    tokens = preprocess(text)
    vecs = [embeddings_index[word] for word in tokens if word in embeddings_index]
    if len(vecs) == 0:
        return np.zeros(dim)  # fallback para documentos sem palavras conhecidas
    return np.mean(vecs, axis=0)

In [12]:
# Caminho do arquivo GloVe baixado e descompactado
glove_path = getDriveStorage("glove.6B.100d.txt")

# Carregar embeddings em memória
embeddings_index = {}
with open(glove_path, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = vector
print("Total de palavras no vocabulário GloVe:", len(embeddings_index))

Total de palavras no vocabulário GloVe: 400000


In [ ]:
# Gerar embeddings
X_train = np.vstack([document_vector(text, embeddings_index, 100)
                     for text in tqdm(df_train["text"])])

y_train = df_train["label"].values

print("Shape final:", X_train.shape, y_train.shape)


100%|████████████████████████████████████████████████████████████████████████████| 3340/3340 [00:02<00:00, 1612.12it/s]

Shape final: (3340, 100) (3340,)


In [ ]:
# salvar embeddings e labels
np.save("data/processed/X_train_fnd_glove100.npy", X_train)
np.save("data/processed/y_train_fnd.npy", y_train)

In [ ]:
# para o teste também
X_test = np.vstack([document_vector(text, embeddings_index, 100)
                    for text in df_test["text"]])
y_test = df_test["label"].values

In [ ]:
np.save("data/processed/X_test_fnd_glove100.npy", X_test)
np.save("data/processed/y_test_fnd.npy", y_test)

In [8]:
# reabrir
X_train = np.load(getDriveStorage("processed/X_train_fnd_glove100.npy"))
y_train = np.load(getDriveStorage("processed/y_train_fnd.npy"))
X_test = np.load(getDriveStorage("processed/X_test_fnd_glove100.npy"))
y_test = np.load(getDriveStorage("processed/y_test_fnd.npy"))

## Embedding

In [13]:
# Carregar GloVe já no formato word2vec (se não fez antes, converter)
glove_file = getDriveStorage("glove.6B.100d.txt")
tmp_file = getDriveStorage("glove.6B.100d.word2vec.txt")

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

In [14]:
# Agora carregamos como modelo gensim
glove_model = KeyedVectors.load_word2vec_format(tmp_file, binary=False)

### Word Mover’s Distance

In [11]:
from annoy import AnnoyIndex

def wmdistance(docs, embeddings, n, k=3, n_neighbors_ann=50):
    """
    Optimized WMD calculation using Approximate Nearest Neighbor (ANN) search.

    Args:
        docs (list): List of tokenized documents.
        embeddings (np.ndarray): Array of document embeddings (e.g., average GloVe embeddings).
        n (int): Number of documents to process.
        k (int): Number of nearest neighbors to find using WMD.
        n_neighbors_ann (int): Number of approximate nearest neighbors to find using ANN.

    Returns:
        torch.Tensor: Edge index in PyG format.
    """
    edges = []
    vector_size = embeddings.shape[1]

    # Build ANN index
    ann_index = AnnoyIndex(vector_size, 'angular')  # Using angular distance for normalized embeddings
    for i in range(n):
        ann_index.add_item(i, embeddings[i])
    ann_index.build(10)  # 10 trees

    for i in tqdm(range(n)):
        # Find approximate nearest neighbors using ANN
        nearest_neighbors_ann = ann_index.get_nns_by_item(i, n_neighbors_ann + 1) # +1 to include the item itself
        nearest_neighbors_ann = [nn for nn in nearest_neighbors_ann if nn != i] # Exclude the item itself

        dists = []
        # Calculate WMD only for approximate nearest neighbors
        for j in nearest_neighbors_ann:
            try:
                d = glove_model.wmdistance(docs[i], docs[j])
                dists.append((j, d))
            except Exception as e:
                # Handle potential errors during WMD calculation (e.g., out-of-vocabulary words)
                # print(f"Error calculating WMD between doc {i} and {j}: {e}")
                continue


        # Sort by distance and get top-k
        dists = sorted(dists, key=lambda x: x[1])[:k]
        for j, d in dists:
            edges.append((i, j))

    # Bidirectional edges
    edges = edges + [(j,i) for (i,j) in edges]
    # edge_index in PyG format
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edge_index

# To use this optimized function, you would first need document embeddings.
# A simple approach is to average the word embeddings for each document.
# Example of how to get average embeddings (assuming docs is the list of tokenized documents):
def get_avg_embeddings(docs, model, vector_size):
    embeddings = []
    for doc in docs:
        word_embeddings = [model[word] for word in doc if word in model]
        if word_embeddings:
            embeddings.append(np.mean(word_embeddings, axis=0))
        else:
            embeddings.append(np.zeros(vector_size)) # Handle empty documents or documents with only OOV words
    return np.array(embeddings)

print("Optimized wmdistance function and helper function defined.")

Optimized wmdistance function and helper function defined.


In [10]:
len(df_train["text"])

3340

In [ ]:
k = 3

In [ ]:
# calcular o edge index do treino com wmdistance
#O WMD funciona melhor se os textos forem tokenizados.
docs = [preprocess(t) for t in df_train["text"].tolist()]
edge_index_train = wmdistance(docs,embeddings_index, len(docs), k)
print(edge_index_train.shape)  # (2, num_edges)

# edge_index é um tensor torch (2, num_edges)
np.save(getDriveStorage(f"processed/edge_index_train_fnd_k{k}.npy"), edge_index_train.numpy())

In [ ]:
# para test.
docs_test = [preprocess(t) for t in df_test["text"].tolist()]
edge_index_test = wmdistance(docs_test,embeddings_index, len(docs), 3)
np.save(getDriveStorage(f"processed/edge_index_test_fnd_k{k}.npy"), edge_index_test.numpy())

In [ ]:
# carregar os edege index calculados com wmdistance
# carregar o edge_index
edge_index_train = torch.from_numpy(np.load(getDriveStorage(f"processed/edge_index_train_fnd_k{k}.npy")))
edge_index_test = torch.from_numpy(np.load(getDriveStorage(f"processed/edge_index_test_fnd_k{k}.npy")))

### Similaridade de Cosseno

In [ ]:
def build_graph_cosine_weighted(X, k=3):
    sim = cosine_similarity(X)
    N = sim.shape[0]
    edges, weights = [], []

    for i in tqdm(range(N)):
        idx = np.argsort(-sim[i])[1:k+1]    # top-k (exclui i)
        for j in idx:
            edges += [(i, j), (j, i)]
            w = sim[i, j]
            weights += [w, w]

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_weight = torch.tensor(weights, dtype=torch.float)
    # Remove duplicatas e garante alinhamento
    edge_index, edge_weight = coalesce(edges, weights, num_nodes=N, reduce="mean")
    return edge_index, edge_weight

In [ ]:
X_all = np.vstack([X_train, X_test])
k = 3

In [ ]:
edge_index, edge_weight = build_graph_cosine_weighted(X_all, k=k)

np.save(getDriveStorage(f"processed/edge_index_x_all_w_k{k}.npy"), edge_index.numpy())
np.save(getDriveStorage(f"processed/edge_weight_x_all_w_k{k}.npy"), edge_weight.numpy())

In [ ]:
labels = (pd.concat([df_train, df_test], ignore_index=True))["label"].values
np.save(getDriveStorage("processed/labels.npy"), labels)

In [ ]:
labels = np.load(getDriveStorage("processed/labels.npy"))

## Grafo de Similaridade

In [ ]:
N = len(labels)
train_size = len(df_train)
test_size = len(df_test)

train_mask = torch.zeros(N, dtype=torch.bool)
test_mask = torch.zeros(N, dtype=torch.bool)
val_mask = torch.zeros(N, dtype=torch.bool)

# Usa 10% do treino como validação
val_size = int(train_size * 0.1)

train_mask[:train_size-val_size] = True
val_mask[train_size-val_size:train_size] = True
test_mask[train_size:] = True

In [ ]:
# Supondo que você já tem:
# X_all: np.array (N, 100) com embeddings de todos os documentos
# labels: np.array (N,) com rótulos 0/1
# edge_index: tensor (2, num_edges) com arestas
# train_mask, val_mask, test_mask: tensores booleanos

# Converter para tensores
scaler = StandardScaler()
X_all = scaler.fit_transform(X_all)
x = torch.tensor(X_all, dtype=torch.float)
y = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)

# Adicionar máscaras
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
data.edge_weight = edge_weight

print(data)

## GCN BN

In [ ]:
class GCN_BN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout
    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        return x

In [ ]:
# Configurações
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GCN_BN(in_channels=data.num_node_features,
            hidden_channels=64,
            out_channels=data.y.max().item()+1,
            dropout=0.5).to(device)

data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
def evaluate_metrics():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1).cpu().numpy()
    true = data.y.cpu().numpy()

    results = {}
    for name, mask in zip(["train", "val", "test"],
                          [data.train_mask, data.val_mask, data.test_mask]):
        mask = mask.cpu().numpy()
        y_true, y_pred = true[mask], pred[mask]

        acc = (y_true == y_pred).mean()
        f1 = f1_score(y_true, y_pred, average="macro")
        precision = precision_score(y_true, y_pred, average="macro")
        recall = recall_score(y_true, y_pred, average="macro")

        results[name] = {
            "acc": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }
        # Para o conjunto de teste, retornar também os scores
        if name == "test":
            results[name]['scores'] = out[mask].cpu().detach().numpy()
    return results


### Treino

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

labels = data.y.cpu().numpy()
idx_all = np.arange(len(labels))

all_metrics = []
history_metrics = {}
y_np = data.y.cpu().numpy()
classes = np.unique(y_np)
cw = compute_class_weight('balanced', classes=classes, y=y_np)
class_weights = torch.tensor(cw, dtype=torch.float, device=device)

for fold, (train_idx, test_idx) in enumerate(kf.split(idx_all)):
    start_time = time.time()  # início do treino do fold
    # Máscaras
    train_mask = torch.zeros(len(labels), dtype=torch.bool)
    test_mask = torch.zeros(len(labels), dtype=torch.bool)
    val_mask = torch.zeros(len(labels), dtype=torch.bool)

    # separa validação (10% do treino)
    val_size = int(0.1 * len(train_idx))
    val_idx = train_idx[:val_size]
    train_idx = train_idx[val_size:]

    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True

    # Atualiza masks no objeto data
    data.train_mask = train_mask.to(device)
    data.val_mask = val_mask.to(device)
    data.test_mask = test_mask.to(device)

    # Update edge_index and edge_weight in the data object for each fold
    data.edge_index = edge_index.to(device)
    data.edge_weight = edge_weight.to(device)

    # Novo modelo por fold
    model = GCN_BN(in_channels=data.num_node_features,
                hidden_channels=32,
                out_channels=data.y.max().item()+1,
                dropout=0.5).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

    # Treino
    best_val_f1 = 0
    best_metrics = None
    history_metrics[fold] = []

    for epoch in range(1, 300):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        metrics = evaluate_metrics()
        history_metrics[fold].append(metrics.copy())
        if metrics["val"]["f1"] > best_val_f1:
            best_val_f1 = metrics["val"]["f1"]
            best_metrics = metrics

    metrics = best_metrics["test"]
    metrics["train_time"] = time.time() - start_time
    all_metrics.append(metrics)
    print(f"Fold {fold+1} | Test F1: {metrics['f1']:.4f} | "
          f"Acc: {metrics['acc']:.4f} | Tempo: {metrics['train_time']:.2f}s")


### Métricas

In [ ]:
# Média e desvio padrão das métricas
# Excluir 'scores' da lista de métricas para cálculo de média e desvio padrão
metrics_to_average = [m for m in all_metrics[0] if m != 'scores']

avg_metrics = {m: np.mean([fold[m] for fold in all_metrics]) for m in metrics_to_average}
std_metrics = {m: np.std([fold[m] for fold in all_metrics]) for m in metrics_to_average}

print(f"\nResultados Médios (5-Fold Cross Validation) para k = {k}:")
for metric in avg_metrics:
    print(f"{metric.capitalize()} | Média: {avg_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")

In [ ]:
# Criar uma lista vazia para armazenar os dados de cada fold
all_history_data = []

# Iterar sobre cada fold no dicionário history_metrics
for fold_id, fold_data in history_metrics.items():
    # Para cada dicionário na lista do fold, adicionar o ID do fold
    for epoch_data in fold_data:
        epoch_data['fold'] = fold_id
        all_history_data.append(epoch_data)

# Converter a lista de dicionários para um DataFrame
df_hm = pd.DataFrame(all_history_data)

# Expandir as colunas de dicionário (train, val, test)
df_hm = pd.json_normalize(df_hm.to_dict('records'))
df_hm.columns = df_hm.columns.str.replace('.', '_', regex=False)

#print(df_hm.head())
df_hm

In [ ]:
df_hm.to_csv(getDriveStorage(f"outputs/gcnbnk{k}_fnd_crossval_history.csv"))
df_hm.to_excel(getDriveStorage(f"outputs/gcnbnk{k}_fnd_crossval_history.xlsx"))

In [ ]:
# Exemplo: salvar o modelo do último fold
torch.save(model.state_dict(), getDriveStorage(f"outputs/gcnbnk{k}_fnd_model_fold5.pth"))

In [ ]:
# Para carregar depois:
model = GCN_BN(in_channels=data.num_node_features,
            hidden_channels=64,
            out_channels=data.y.max().item()+1,
            dropout=0.5)
model.load_state_dict(torch.load(getDriveStorage(f"outputs/gcnbnk{k}_fnd_model_fold5.pth")))
model.eval()

In [ ]:
# Converte lista de dicts em DataFrame
df_results = pd.DataFrame(all_metrics)

# Excluir 'scores' da lista de métricas para cálculo de média e desvio padrão
metrics_to_average = [m for m in df_results.columns if m != 'scores']

# Calcula médias e desvios
mean_metrics = df_results[metrics_to_average].mean().to_dict()
std_metrics = df_results[metrics_to_average].std().to_dict()

# Adiciona linhas "Mean" e "Std"
df_results.loc["Mean", metrics_to_average] = mean_metrics.values()
df_results.loc["Std", metrics_to_average] = std_metrics.values()

# Excluir a coluna 'scores' antes de salvar
df_results_to_save = df_results.drop(columns=['scores'])

# Salva em CSV e Excel
df_results_to_save.to_csv(getDriveStorage(f"outputs/gcnbnk{k}_fnd_crossval_results.csv"))
df_results_to_save.to_excel(getDriveStorage(f"outputs/gcnbnk{k}_fnd_crossval_results.xlsx"))

print(df_results)